In [89]:
import numpy as np
import tensorflow as tf
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Conv3D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import keras
from keras import Input, layers
from keras.models import Model
import keras.backend as K
import PIL
from PIL import Image
from numpy import asarray
import urllib.request
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
from io import BytesIO
from keras.preprocessing import image
from tensorflow.python.ops.numpy_ops import np_config
import cv2
import networkx as nx
import json
from networkx.generators.small import krackhardt_kite_graph 
from string import ascii_lowercase 
import io
import base64
from skimage.io import imread
import os

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['imread']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [113]:
all_imgs = load_images_from_folder('/content/drive/MyDrive/ct_covid')

In [79]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [126]:
e = 0
for i in all_imgs:
  x = tf.convert_to_tensor(i, dtype=tf.float32)
  x /= 255
  dim = x.shape
  x = x.reshape(dim[0], dim[1], dim[2], 1)
  #print(x.shape)
  #print(x)
  x = np.array(x)
  kernel_1 = np.array([
            [[[0]], [[-1]], [[0]]],
            [[[-1]], [[5]], [[-1]]],
            [[[0]], [[-1]], [[0]]]
        ])
  x = tf.constant(x, dtype=tf.float32)
  kernel = tf.constant(kernel_1, dtype=tf.float32)
  out_1 = tf.nn.conv2d(x, kernel, strides=[2], padding='VALID')
  out_1 = tf.squeeze(out_1)
  out_1 = out_1.numpy()
  tensor_to_image(out_1)
  am = (np.dot(out_1, out_1.T) > 0).astype(int)
  D = nx.DiGraph(am)
  nodes = list(D.nodes)
  edges = list(D.edges)
  G = nx.Graph()
  G.add_nodes_from(nodes)
  G.add_edges_from(edges)
  e = e + 1
  nx.write_gexf(G, "%s.gexf" %e)